In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
from scipy.misc import imread
from scipy.io import loadmat
import os, sys
import glob

In [ ]:
# load a picture of the empty track
track = loadmat('track.mat')
track = track['video']
track = track[13, 100:457, :480, :]
track2 = track.copy()

vids = glob.glob('*.mat')

for vid in vids:
    # load the test video and do some pre-processing
    videofile = loadmat(vid)
    videofile = videofile['video']
    videofile = np.mean(videofile[40:, 100:457, :480, :], 3)

    oldm = 0
    oldn = 0
    steps = 0


    for i in range(videofile.shape[0]-1):
        diff = videofile[i, ...] - videofile[i+1, ...]
        diff = np.uint8(diff > 16) * 255

        contours, _ = cv2.findContours(diff.copy(), 1, 2)
        c = max(contours, key = cv2.contourArea)
        x,y,w,h = cv2.boundingRect(c)
        m = np.int32(y+h/2)
        n = np.int32(x+w/2)
    
    #track[m-1:m+1, n-1:n+1, 1] = 255
    
        if oldm != 0:
            cv2.line(track2, (oldn, oldm), (n, m), (0, 255, 0))
        else: first_m, first_n = m, n
        
        if np.abs(m-first_m) <= 10 and np.abs(n - first_n) <= 15:
            if steps > 20:
                break
        
        oldm, oldn = m, n
        steps += 1


fig = plt.figure(figsize=[16, 16])
a2 = fig.add_subplot(111)
a2.imshow(track2)
a2.axis('off')
plt.show()